Here we estimate the following conditional median function:
$$
\text{med}[valuation_i|ispolice_i, sellerfeedbackscore_i]=\alpha+\beta_1 ispolice_i+\beta_2 sellerfeedbackscore_i
$$
using the entire sample where the number of bids received is strictly between 3 and 12.

In [1]:
import sys
import pandas as pd
 
sys.path.append('../')

from functions import *

data = pd.read_csv("../../data/demeaned.csv")
df = data.groupby(["id", "ispolice", "sellerfeedbackscore", "bidcount", "apple", "amazon", "increment_residual"])["residual"].apply(lambda x: x.values).reset_index()

/Users/liumukun/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
valid_bids = list(df[(df.ispolice == 1) & (df.apple == 1)].bidcount.value_counts().index)
include = df[(df.bidcount > 1) & (df.bidcount.isin(valid_bids)) & (df.apple == 1)]

bids = list(include.residual)

logged_feedback = np.log(include.sellerfeedbackscore+1)
logged_feedback = transform_covariates(logged_feedback, 100)
include.sellerfeedbackscore = logged_feedback

covariates = np.array(include[["ispolice", "sellerfeedbackscore"]])
covariates = list([list(cov) for cov in covariates])
incremented = list(include.increment_residual)

<ipython-input-2-72b2804822f4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  include.sellerfeedbackscore = logged_feedback


In [ ]:
median_upper, median_lower = estimate_median(bids, covariates, incremented, (-4,6))
dump(median_lower, median_upper, "CMF_apple")

calculating values for covariate: [1.0, 10.094912597722205] (1/99)
total time elapsed: 0.0001593980000000883s
calculating values for covariate: [0.0, 2.05067072014674] (2/99)
total time elapsed: 121.867780628s
calculating values for covariate: [0.0, 6.410962598537746] (3/99)
total time elapsed: 242.443792789s


In [ ]:
cef1 = lambda c, cov: c[0]*(1-cov[0])+c[1]*cov[0]+c[2]*cov[1]*(1-cov[0])+c[3]*cov[1]*cov[0]
n_cov1 = 4

In [ ]:
def loss_function1(c):
    cef = lambda cov: cef1(c, cov)
    return get_loss_function(covariates, median_upper, median_lower, cef)

point1, interval1 = get_estimates(loss_function1, n_cov1)

print(point1)
print(interval1)

In [ ]:
bootstrap(bids, covariates, incremented, [n_cov1], [cef1], 30, "apple")